# Testing

We will spend a lot of time writing algorithms that do some numerical task. We will spend *much* more time fixing errors we've made and checking that they work as expected. This is testing: the constant checking that everything is doing what we want.

## Fixing code

Take a look at the file `lab3_testinverse.py`. The full contents define a single function:

In [1]:
# This script requires an intro comment
def test_inverse(A, tol):
    """
    This function requires documentation.
    """
    import numpy

    invtol = 1.0 / tol

    assert(numpy.linalg.cond(A) < tol), 'The condition number is too big!'
    assert(numpy.linalg.det(A < invtol), 'The determinant of the matrix is too small!'

    inverseA = numpy.linalg.inv(A)

    return inverseA

SyntaxError: invalid syntax (<ipython-input-1-974bac3c1b47>, line 13)

If you open this in Spyder, or run it as above, you see that there is a syntax error. In fact there are a number of errors. Some of these can be picked up automatically by Spyder. Others need more thought and analysis.

### Spyder and static code analysis

Open the file in Spyder's editor. Look down the left bar of the editor. You should see a big red cross next to the line starting `InverseA ...`. Hovering over that tells you that it's a syntax error.

Syntax errors appear at the first place that Python can't work out what's going on. Sometimes it appears directly in the line where there's a problem. Sometimes, particularly when the brackets go wrong, the error is actually in a previous line. The error analysis thinks that the previous line is continuing because the brackets didn't close, and shows an error on the next line.

That's the problem in this case: the second `assert` statement hasn't correctly closed the bracket. We should fix that.

In [15]:
# This script requires an intro comment
def test_inverse(A, tol):
    """
    This function requires documentation.
    """
    import numpy

    invtol = 1.0 / tol

    assert(numpy.linalg.cond(A) < tol), 'The condition number is too big!'
    assert(numpy.linalg.det(A < invtol)), 'The determinant of the matrix is too small!'

    inverseA = numpy.linalg.inv(A)

    return inverseA

### Known solutions

When starting to test we should always use problems that we know the answer to, and are as simple as possible. For example, the identity matrix should invert to itself. We also know that the condition number of the identity matrix is $1$, so we can use a tolerance of, say, $2$ without problems. Let's try that:

In [16]:
import numpy
I2 = numpy.eye(2)
test_inverse(I2, 2)

array([[1., 0.],
       [0., 1.]])

This works, and produces the correct answer. Next we should try a simple matrix. For example, we know that
$$
A = \begin{pmatrix} 1 & -1 \\ 1 & 1 \end{pmatrix} \quad \implies \quad A^{-1} = \frac{1}{2} \begin{pmatrix} 1 & 1 \\ -1 & 1 \end{pmatrix}.
$$

In [17]:
A = numpy.array([[1, -1], [1, 1]])
test_inverse(A, 2)

[[ True False]
 [ True  True]]


AssertionError: The determinant of the matrix is too small!

What has gone wrong here? The error message shows a problem with the second `assert` statement. Now, we can compute the determinant of $A$ by hand - it is $2$. With a tolerance of $2$, the inverse tolerance is $1/2$. Clearly $2$ is not less than $1/2$!

Let's pull the code out from the `assert` statement:

In [19]:
invtol = 1/2
numpy.linalg.det(A < invtol)

0.0

Now that we have pulled this out, it's clearer that the brackets are in the wrong place. The test *should* be more like:

In [20]:
numpy.linalg.det(A) < invtol

False

Let's fix that in the code, and then apply some thought:

In [21]:
# This script requires an intro comment
def test_inverse(A, tol):
    """
    This function requires documentation.
    """
    import numpy

    invtol = 1.0 / tol

    assert(numpy.linalg.cond(A) < tol), 'The condition number is too big!'
    assert(numpy.linalg.det(A) < invtol), 'The determinant of the matrix is too small!'

    inverseA = numpy.linalg.inv(A)

    return inverseA

### Documentation

There should now be no errors shown by Spyder. That doesn't mean the code is correct: it just means it can be run. Before we go further, we should first be clear as to what it's meant to do, by fixing the documentation. The crucial thing is to complete the docstring:

In [3]:
# This function should safely invert a matrix, by checking its condition number
# and determinant
def test_inverse(A, tol):
    """
    Invert the matrix, assuming the condition number is smaller than the
    tolerance and the determinant is bigger than the inverse of the
    tolerance.

    Parameters
    ----------

    A : array of float
        matrix to be inverted
    tol : float
        tolerance

    Returns
    -------

    Inverse A : array of float
        inverse of A

    Notes
    -----

    This function relies heavily on numpy to do all real calculations.
    """
    import numpy

    invtol = 1.0 / tol

    assert(numpy.linalg.cond(A) < tol), 'The condition number is too big!'
    assert(numpy.linalg.det(A) < invtol), 'The determinant of the matrix is too small!'

    inverseA = numpy.linalg.inv(A)

    return inverseA

The explicit description in the docstring says that the condition number must be *smaller* than the tolerance, and the matrix determinant *bigger* than the inverse of the tolerance. Looking at the code, we can see that both `assert` statements use the same inequality, `<`. Therefore one must be wrong, and again it's the second. Let's fix that:

In [22]:
# This function should safely invert a matrix, by checking its condition number
# and determinant
def test_inverse(A, tol):
    """
    Invert the matrix, assuming the condition number is smaller than the
    tolerance and the determinant is bigger than the inverse of the
    tolerance.

    Parameters
    ----------

    A : array of float
        matrix to be inverted
    tol : float
        tolerance

    Returns
    -------

    Inverse A : array of float
        inverse of A

    Notes
    -----

    This function relies heavily on numpy to do all real calculations.
    """
    import numpy

    invtol = 1.0 / tol

    assert(numpy.linalg.cond(A) < tol), 'The condition number is too big!'
    assert(numpy.linalg.det(A) > invtol), 'The determinant of the matrix is too small!'

    inverseA = numpy.linalg.inv(A)

    return inverseA

With that fixed, we should now be able to check the matrix from before:

In [23]:
A = numpy.array([[1, -1], [1, 1]])
test_inverse(A, 2)

array([[ 0.5,  0.5],
       [-0.5,  0.5]])

### Compare to an oracle

We have used the `numpy` matrix inversion here. However, if we *had* written our own code, then we could (and should) use such a routine to cross-check. We could then construct a random matrix and check that our code and the oracle code produce the same, or sufficiently similar, results.

## Breakpoints

Sometimes looking at the code and its results is not enough to spot the problem. Instead you need information as to what it's doing as it steps through long loops or complex branches.

Download the `lab3_testsequence.py` file and open in Spyder:

In [25]:
def test_sequence(N):
    """
    Compute the infinite sum of 2^{-n} starting from n = 0, truncating
    at n = N, returning the value of 2^{-n} and the truncated sum.

    Parameters
    ----------

    N : int
        Positive integer, giving the number of terms in the sum

    Returns
    -------

    limit : float
        The value of 2^{-N}
    sumseq : float
        The value of the truncated sum

    Notes
    -----

    The limiting value should be zero, and the value of the sum should
    converge to 2.
    """

    # Start sum from zero, so give zeroth term
    limit = 1.0
    sumseq = 1.0

    # At each step, increment sum and change summand
    for i in range(1, N+1):
        sumseq += limit
        limit /= 2.0

    return limit, sumseq

print(test_sequence(50))

(8.881784197001252e-16, 2.9999999999999982)


This should show no errors, has clear documentation, but the result is wrong: the limit it is meant to compute (the second output) should be $2$, but the result is converging to $3$.

In Spyder, you can set a breakpoint by clicking next to the line numbers on the left. Click next to any line inside the function, such as the first line inside the loop (`sumseq += limit`). Then, from the `Debug` menu, choose `Run`.

This should run the function up to the line with the breakpoint. It stops the function before executing that line. You can now check the `Variable Inspector` to see what the values of the variables are. You can step through the code one line at a time.

By comparing what the function is *actually doing* for the first couple of steps, against how you think the sum should *actually behave*, you should see that the problem is the order of operations inside the loop. Swap those two lines and the result will be correct.

## Automated testing

Automated testing is a very useful way of checking that our code behaves as expected. Let's illustrate that by solving the matrix problem $A {\bf x} = {\bf b}$.

## Gauss Elimination

In lectures we saw the Gaussian Elimination algorithm. This takes the augmented matrix and applies row operations until the matrix is in diagonal form, and then uses back substitution to find the solution.

Let's write that out in code:

In [1]:
import numpy

def gauss_elimination(A, b):
    """
    Solve A x = b
    """
    n = len(b)
    aug = numpy.hstack((A, b))
    # Elimination
    for col in range(n):
        for row in range(col+1,n):
            pivot = aug[row,col] / aug[col,col]
            aug[row,:] -= pivot * aug[col,:]
    # Back substitution
    x = numpy.zeros_like(b)
    for row in range(n-1,-1,-1):
        x[row] = aug[row, -1] / aug[row, row]
        for col in range(row+1,n):
            x[row] -=  aug[row, col] * x[col] / aug[row, row]
    return x

So, does this work? To check it, let's look at the simplest case:

$$
A = \begin{pmatrix} 1 & 0 \\ 0 & 1 \end{pmatrix}, \quad {\bf b} = \begin{pmatrix} 1 \\ 1 \end{pmatrix} \quad \implies \quad {\bf x} = {\bf b}.
$$

In [2]:
A = numpy.identity(2)
b = numpy.ones((2,))
print(gauss_elimination(A, b))

ValueError: all the input arrays must have same number of dimensions

This test shows up an annoying error with our code: it doesn't want to form the augmented matrix. Let's print out what the shape of the various arrays are:

In [3]:
print(A.shape)
print(b.shape)

(2, 2)
(2,)


To glue together the two arrays we need it to see that ${\bf b}$ is equivalent to a column vector. We can do this using `reshape`:

In [4]:
print(numpy.reshape(b, (2,1)))
print(numpy.hstack((A, numpy.reshape(b, (2,1)))))

[[ 1.]
 [ 1.]]
[[ 1.  0.  1.]
 [ 0.  1.  1.]]


So we now correct our code:

In [5]:
import numpy

def gauss_elimination(A, b):
    """
    Solve A x = b
    """
    n = len(b)
    aug = numpy.hstack((A, numpy.reshape(b, (n, 1))))
    # Elimination
    for col in range(n):
        for row in range(col+1,n):
            pivot = aug[row,col] / aug[col,col]
            aug[row,:] -= pivot * aug[col,:]
    # Back substitution
    x = numpy.zeros_like(b)
    for row in range(n-1,-1,-1):
        x[row] = aug[row, -1] / aug[row, row]
        for col in range(row+1,n):
            x[row] -=  aug[row, col] * x[col] / aug[row, row]
    return x

In [6]:
print(gauss_elimination(A, b))

[ 1.  1.]


We now see that this is the same as the input, but we should check that in code:

In [7]:
print(b == gauss_elimination(A, b))

[ True  True]


This checks every component individually, and will be painful to check for large vectors. We can check if *all* entries are the same, which is all we care about:

In [8]:
print(numpy.all(b == gauss_elimination(A, b)))

True


So this test works.

Let us try something more interesting:

$$
A = \begin{pmatrix} 1 & 2 \\ 3 & 4 \end{pmatrix}, \quad {\bf b} = \begin{pmatrix} 5 \\ 6 \end{pmatrix} \quad \implies \quad {\bf x} = \begin{pmatrix} -4 \\ 9/2 \end{pmatrix}.
$$

In [9]:
A = numpy.array([[1.0, 2.0], [3.0, 4.0]])
b = numpy.array([5.0, 6.0])
print(gauss_elimination(A, b))

[-4.   4.5]


Again this has worked, but we want to check this in code. We specify the exact solution and check:

In [10]:
x_exact = [-4, 4.5]
print(numpy.all(x_exact == gauss_elimination(A, b)))

True


Next we will try a nastier case:

$$
A = \begin{pmatrix} 10^{-20} & 1 \\ 1 & 1 \end{pmatrix}, \quad {\bf b} = \begin{pmatrix} 1 \\ 2 \end{pmatrix} \quad \implies \quad {\bf x} \simeq \begin{pmatrix} 1 \\ 1 \end{pmatrix}.
$$

In [11]:
A = numpy.array([[1.0e-20, 1.0], [1.0, 1.0]])
b = numpy.array([1.0, 2.0])
print(numpy.linalg.solve(A, b))
print(gauss_elimination(A, b))

[ 1.  1.]
[ 0.  1.]


This has failed. This is our classic case where we need to pivot or we have a catastrophic loss of precision. So now we need to change our code. How do we ensure that our code continues to work on the original tests while we're fixing the problem?

## Test functions

We're going to use an automated test runner to do this. With one command we will run all the tests we have available and see which ones pass and which fail. But first, we need to write test functions.

A test function is a function whose name starts with `test_`. It checks if another bit of code is working correctly, by *asserting* that its behaviour does what is expected. Here are the test functions corresponding to the tests above:

In [12]:
def test_diagonal():
    A = numpy.identity(2)
    b = numpy.ones((2,))
    assert(numpy.all(b == gauss_elimination(A, b)))
    
def test_full():
    A = numpy.array([[1.0, 2.0], [3.0, 4.0]])
    b = numpy.array([5.0, 6.0])
    x_exact = [-4.0, 4.5]
    assert(numpy.all(x_exact == gauss_elimination(A, b)))
    
def test_pivoting():
    A = numpy.array([[1.0e-20, 1.0], [1.0, 1.0]])
    b = numpy.array([1.0, 2.0])
    x_exact = [1.0, 1.0]
    assert(numpy.all(x_exact == gauss_elimination(A, b)))

Let us run the tests and see what happens:

In [13]:
test_diagonal()

In [14]:
test_full()

In [15]:
test_pivoting()

AssertionError: 

If the test works, *nothing happens*. If the test fails, it tells you that it did.

This is useful, but we still have to remember to execute all the tests. We could make a script to do this, but there's a better way.

## `py.test`

We want to run *all* tests. To do this we'll need to save our code to a script. So create a file (let's call it `gauss_elimination.py` containing our code so far:

```python
import numpy

def gauss_elimination(A, b):
    """
    Solve A x = b
    """
    n = len(b)
    aug = numpy.hstack((A, numpy.reshape(b, (n, 1))))
    # Elimination
    for col in range(n):
        for row in range(col+1,n):
            pivot = aug[row,col] / aug[col,col]
            aug[row,:] -= pivot * aug[col,:]
    # Back substitution
    x = numpy.zeros_like(b)
    for row in range(n-1,-1,-1):
        x[row] = aug[row, -1] / aug[row, row]
        for col in range(row+1,n):
            x[row] -=  aug[row, col] * x[col] / aug[row, row]
    return x

def test_diagonal():
    A = numpy.identity(2)
    b = numpy.ones((2,))
    assert(numpy.all(b == gauss_elimination(A, b)))
    
def test_full():
    A = numpy.array([[1.0, 2.0], [3.0, 4.0]])
    b = numpy.array([5.0, 6.0])
    x_exact = [2, 1.5]
    assert(numpy.all(x_exact == gauss_elimination(A, b)))
    
def test_pivoting():
    A = numpy.array([[1.0e-10, 0.0], [1.0, 1.0]])
    b = numpy.array([1.0, 2.0])
    x_exact = [1.0e10, -1.0e10]
    assert(numpy.all(x_exact == gauss_elimination(A, b)))
```

Now, in the console, type

```python
import pytest
pytest.main("-x gauss_elimination.py")
```

This command will run all of the test functions in the `gauss_elimination.py` file.

You should see output looking like

```bash
collected 3 items 

gauss_elimination.py ..F

=========================================== FAILURES ===========================================
________________________________________ test_pivoting _________________________________________

    def test_pivoting():
        A = numpy.array([[1.0e-10, 1.0], [1.0, 1.0]])
        b = numpy.array([1.0, 2.0])
        x_exact = [1.0, 1.0]
>       assert(numpy.all(x_exact == gauss_elimination(A, b)))
E       assert <function all at 0x1034fa378>([1.0, 1.0] == array([ -1.00000000e+10,   2.00000000e+00])
E        +  where <function all at 0x1034fa378> = numpy.all
E         Use -v to get the full diff)

gauss_elimination.py:37: AssertionError
============================== 1 failed, 2 passed in 0.31 seconds ==============================
```

This tells us that it

* found 3 tests (`collected 3 items`)
* 1 failed and 2 passed (final line, or the `..F`: each `.` is a passing test)
* the test that failed is the one we expected.

Now, when we change the code, we can run all the tests to make sure we didn't break anything.

## Close enough?

We can now try and change the code to include pivoting. Modify your `gauss_elimination` function to read

```python
...
    # Elimination
    for col in range(n):
        # Find the location of the pivot
        ind = numpy.argmax(numpy.abs(aug[col:, col]))
        if ind != col:
            # One liner to swap the rows; think carefully!
            aug[[col,ind+col],:] = aug[[ind+col, col],:]
        for row in range(col+1,n):
            pivot = aug[row,col] / aug[col,col]
            aug[row,:] -= pivot * aug[row,:]
...
```

This will now perform (partial) pivoting. We can now immediately run our tests again by typing

```python
pytest.main("-x gauss_elimination.py")
```

and we should see something like

```bash
collected 3 items 

gauss_elimination_with_pivoting.py .FF

=========================================== FAILURES ===========================================
__________________________________________ test_full ___________________________________________

    def test_full():
        A = numpy.array([[1.0, 2.0], [3.0, 4.0]])
        b = numpy.array([5.0, 6.0])
        x_exact = [-4.0, 4.5]
>       assert(numpy.all(x_exact == gauss_elimination(A, b)))
E       assert <function all at 0x1034f9378>([-4.0, 4.5] == array([-4. ,  4.5])
E        +  where <function all at 0x1034f9378> = numpy.all
E         Use -v to get the full diff)

gauss_elimination.py:36: AssertionError
________________________________________ test_pivoting _________________________________________

    def test_pivoting():
        A = numpy.array([[1.0e-10, 1.0], [1.0, 1.0]])
        b = numpy.array([1.0, 2.0])
        x_exact = [1.0, 1.0]
>       assert(numpy.all(x_exact == gauss_elimination(A, b)))
E       assert <function all at 0x1034f9378>([1.0, 1.0] == array([ 1.,  1.])
E        +  where <function all at 0x1034f9378> = numpy.all
E         Use -v to get the full diff)

gauss_elimination.py:42: AssertionError
============================== 2 failed, 1 passed in 0.32 seconds ==============================
```

Now the middle test has failed as well! However, look at what it is comparing: `([-4.0, 4.5] == array([-4. ,  4.5])`. These appear to be identical, but it's clearly finding a small difference.

In [16]:
def gauss_elimination(A, b):
    """
    Solve A x = b
    """
    n = len(b)
    aug = numpy.hstack((A, numpy.reshape(b, (n, 1))))
    # Elimination
    for col in range(n):
        # Find the location of the pivot
        ind = numpy.argmax(numpy.abs(aug[col:, col]))
        if ind != col:
            # One liner to swap the rows; think carefully!
            aug[[col,ind+col],:] = aug[[ind+col, col],:]
        for row in range(col+1,n):
            pivot = aug[row,col] / aug[col,col]
            aug[row,:] -= pivot * aug[col,:]
    # Back substitution
    x = numpy.zeros_like(b)
    for row in range(n-1,-1,-1):
        x[row] = aug[row, -1] / aug[row, row]
        for col in range(row+1,n):
            x[row] -=  aug[row, col] * x[col] / aug[row, row]
    return x

We can check this explicitly by looking at the result of our latest code in detail:

In [17]:
A = numpy.array([[1.0, 2.0], [3.0, 4.0]])
b = numpy.array([5.0, 6.0])
x_exact = numpy.array([-4.0, 4.5])
x = gauss_elimination(A, b)
print(x - x_exact)

[  8.88178420e-16  -8.88178420e-16]


We see that the difference is truly tiny, and due to floating point errors, not errors in the algorithm. We should modify our check: instead of making certain it is *exactly* equal, we should check if the result is *close*. Thankfully, there is a function for this: `numpy.allclose`. Modify the tests to read

```python
def test_diagonal():
    A = numpy.identity(2)
    b = numpy.ones((2,))
    assert(numpy.allclose(b, gauss_elimination(A, b)))
    
def test_full():
    A = numpy.array([[1.0, 2.0], [3.0, 4.0]])
    b = numpy.array([5.0, 6.0])
    x_exact = [-4.0, 4.5]
    assert(numpy.allclose(x_exact, gauss_elimination(A, b)))
    
def test_pivoting():
    A = numpy.array([[1.0e-10, 1.0], [1.0, 1.0]])
    b = numpy.array([1.0, 2.0])
    x_exact = [1.0, 1.0]
    assert(numpy.allclose(x_exact, gauss_elimination(A, b)))
```

Now we should see

```bash
collected 3 items 

gauss_elimination.py ...

=================================== 3 passed in 0.31 seconds ===================================
```

# Exercises

1. Add more tests. Check higher dimensional matrices.
2. What happens if you put in "incorrect" input? Say you set `b="dog"` rather than an array. Or suppose the matrix and vector have incompatible dimensions. Investigate `pytest.raises` to write tests that check that the function fails the way you expect.
3. When your code fails, is the output easy to understand? If not, add `assert` statements *inside* the `gauss_elimination` function that are more meaningful: for example `assert(len(A) == len(b)), "Matrix A and vector b must have compatible dimensions!"`